In [1]:
from torch import load
import torch
import sys
import numpy as np
from schnetpack.data import AtomsData
import schnetpack as spk
import ase.db
import ase.io
from ase.io import read,write
import joblib
import lightgbm as lgb
import os
from openbabel import openbabel
from rdkit import Chem

In [2]:
def load_model(target,path):
    if target!='PCE':
        model1=load(path, map_location=torch.device('cpu'))
        model1.eval()
        return model1
    else:
        model= joblib.load(path)
        
        return model
    

In [3]:
def cal_nd(mol):
    atoms=mol.toatoms()
    write('mol.xyz',atoms)
    obConversion = openbabel.OBConversion()
    obConversion.SetInAndOutFormats("xyz", "mol")
    mol = openbabel.OBMol()
    obConversion.ReadFile(mol, "mol.xyz")   # Open Babel will uncompress automatically
    mol.AddHydrogens() 
    obConversion.WriteFile(mol, '1.mol')

    #calculate Nd         
    mol = Chem.MolFromMolFile('1.mol')        
    n = len(mol.GetAtoms())         
    Nd = 0        
    for i in range(0,n):
        atom = mol.GetAtomWithIdx(i)
        #判断原子是否为芳香性
        if atom.GetIsAromatic() == True:
            Nd += 1
        if atom.GetIsAromatic() == False:
            #判断原子价电子是否等于总饱和度
            if atom.GetTotalValence() != atom.GetTotalDegree():
                Nd += 1
            if atom.GetTotalValence() == atom.GetTotalDegree():
                #判断原子是否在环上
                if atom.IsInRing() == True:
                    Nd += 1
        
        
    return Nd

In [4]:
def cal_prop(moln,molo,tag):
    
    
    al=.0
    if molo.data.Acceptor=='PC61BM':
        al= -3.70
        adl= 0.077824564
    if molo.data.Acceptor=='PC71BM':
        al= -3.91
        adl= 0.033470005
    if tag=='edahl':
        prop=al-float(moln.homo)
    if tag=='edall':
        prop=float(moln.lumo)-al
    if tag=='adlumo':
        prop=adl
    if tag=='nd':
        prop=cal_nd(moln)


    return prop

In [5]:
def pred_data( model,tag,data):
     
            
    if tag== 'PCE':
        return pred_pce(model,data)
        
    else :
         return pred_prop(model,tag,data)    
             

In [6]:
def pred_pce(model,data):
    db=ase.db.connect(data)
    pce=[]
    ids=[]
    for row in db.select():
        x=[]
        x.extend((row.homo,row.lumo,row.edahl,row.edall,row.et1,row.nd,row.adlumo,row.dhomo,row.dlumo))
        print(x)
        y = model.predict(np.array(x).reshape(1,-1)).tolist()
        print(y)
        pce.extend(y)
        ids.append(row.id)
        
    return ids,pce

In [7]:
def pred_prop(model,tag,data):
    pred=AtomsData(data)
    pred_loader = spk.AtomsLoader(pred, batch_size=10) #40!!
    for count, batch in enumerate(pred_loader):
        datapred = model(batch)
        ids=batch['_idx'].numpy().tolist()
        datapred=datapred[tag].detach().numpy().tolist()
        yield datapred,ids

In [8]:
def write_results(predata,tag,db):
    
    for num in predata.keys():
        for prop in predata[num].keys():
            
            db.update(id=num+1, **{prop: predata[num][prop]}) 
    
    return 0

In [9]:
def main():
    target=['et1','dhomo','dlumo','homo','lumo'] # need to predict with schnet
    target2=['nd','edahl','edall','adlumo'] # no need to predict
    predata={}
    db=ase.db.connect('predCADAC.db')
    
    odb=ase.db.connect('CADAC.db')
    for mol in odb.select():      
            atom=mol.toatoms()
            db.write(atom,name = mol.data.name)
    for tag in target:
        best_model=load_model(target=tag,path='./package/'+tag+'_model')
    
        for property,id in pred_data(best_model,tag,data='predCADAC.db'):
            for sid,sprop in zip(id,property):    
                predata.update({sid[0]:{tag:sprop[0]}})
        write_results(predata,tag,db)    
    
    for tag in target2:
        for moln,molo in zip(db.select(),odb.select()):
            sprop=cal_prop(moln,molo,tag)
            sid=moln.id-1
            predata.update({sid:{tag:sprop}})
           
        write_results(predata,tag,db) 
    pcemodel=load_model(target='PCE',path='./package/lgb_model')
    
    ids,pce=pred_data(model=pcemodel,tag='PCE',data='predCADAC.db')

    for sid,spce in zip(ids,pce):
        
        db.update(id=sid,PCE=spce)

    
    return 0

In [11]:
if __name__ == '__main__':
    status = main()
    
    

[-5.149979591369629, -2.792536735534668, 1.2399795913696288, 1.1174632644653322, 1.291740894317627, 60, 0.033470005, 0.399941086769104, 0.051153700798749924]
[5.876020366399995]
[-4.955236434936523, -2.5697479248046875, 1.0452364349365233, 1.3402520751953126, 1.1825655698776245, 49, 0.033470005, 0.5977578163146973, 0.7276536822319031]
[3.7681095450592332]
[-4.949885845184326, -2.9508652687072754, 1.039885845184326, 0.9591347312927248, 1.3003145456314087, 80, 0.033470005, 0.32938939332962036, 0.21963128447532654]
[6.373613845251848]
[-5.160260200500488, -2.692142963409424, 1.2502602005004881, 1.2178570365905763, 1.4375345706939697, 53, 0.033470005, 0.47087228298187256, 0.2700880765914917]
[5.797928283777412]
[-5.067202568054199, -2.7234015464782715, 1.157202568054199, 1.1865984535217287, 1.4922748804092407, 58, 0.033470005, 0.40772685408592224, 0.21812289953231812]
[5.190808050053835]
[-4.890591621398926, -3.089444160461426, 0.9805916213989256, 0.8205558395385744, 1.0248838663101196, 60

[5.0947724776818575]
[-5.014739036560059, -2.9054248332977295, 1.1047390365600585, 1.0045751667022706, 1.6496702432632446, 66, 0.033470005, 0.32907021045684814, 0.17284591495990753]
[4.816673233429388]
[-5.009326934814453, -2.9979212284088135, 1.099326934814453, 0.9120787715911867, 1.0844639539718628, 64, 0.033470005, 0.5841910243034363, 0.5758014917373657]
[3.634296853528523]
[-4.847745418548584, -2.8698763847351074, 0.9377454185485838, 1.0401236152648927, 0.8701762557029724, 53, 0.033470005, 0.930932879447937, 1.0206713676452637]
[3.045097476789734]
[-4.906632423400879, -3.106422185897827, 0.9966324234008788, 0.803577814102173, 1.1653567552566528, 84, 0.033470005, 0.3662712275981903, 0.5268509387969971]
[6.038809848873149]
[-5.084147930145264, -2.945582628250122, 1.1741479301452635, 0.9644173717498781, 1.168982982635498, 57, 0.033470005, 0.7163110971450806, 0.6475925445556641]
[3.2886031205272195]
[-4.933717250823975, -2.9475584030151367, 1.0237172508239745, 0.9624415969848634, 1.246